# 🍄 MUSHROOM EVALUATION



RAG-Based Mushroom Classifier with Visual Knowledge Base
ใช้ภาพตัวอย่างจาก Knowledge Base เพื่อช่วยในการจำแนก

In [ ]:
# Install & Import
import subprocess
import sys

packages = ["transformers", "torch", "pillow", "pandas", "scikit-learn", "faiss-cpu"]
for pkg in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
import os
import time
import numpy as np
import pickle
import faiss

In [ ]:
# Mount Drive
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    print("Drive mounted\n")
except:
    pass

Mounted at /content/drive
Drive mounted



In [ ]:
# Configuration - SAME AS STREAMLIT
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}\n")

CSV_PATH = "/content/drive/MyDrive/mushroom_dataset_ready.csv"
IMG_DIR = "/content/drive/MyDrive/mushroom_dataset"
KB_CACHE_PATH = "/content/drive/MyDrive/mushroom_knowledge_base.pkl"
OUTPUT_PATH = "/content/drive/MyDrive/evaluation_results_hybrid.csv"

# CRITICAL: Same settings as Streamlit
TOP_K = 3
SAFETY_THRESHOLD = 0.55
MIN_CONFIDENCE = 0.55
NUM_KB_IMAGES = 10

# Load CLIP Model
MODEL_NAME = "openai/clip-vit-large-patch14"
print(f"Loading {MODEL_NAME}...")
model = CLIPModel.from_pretrained(MODEL_NAME).to(device)
processor = CLIPProcessor.from_pretrained(MODEL_NAME)
print("Model loaded\n")

# CLIP Prompts - SAME AS STREAMLIT
EDIBLE_PROMPTS = [
    "a safe edible mushroom suitable for cooking",
    "a choice edible mushroom prized by foragers",
    "an edible Boletus with thick stem and pores",
    "an edible oyster mushroom or similar safe species"
]

POISONOUS_PROMPTS = [
    "a deadly poisonous mushroom that is toxic",
    "a dangerous Amanita mushroom with white gills",
    "a toxic Galerina or Cortinarius species",
    "a poisonous mushroom with warning colors"
]

# Feature Extraction
def extract_image_features(image_path):
    """Extract CLIP features (normalized)"""
    try:
        image = Image.open(image_path).convert('RGB')
        inputs = processor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            features = model.get_image_features(**inputs)
            features = features / features.norm(dim=-1, keepdim=True)

        return features.cpu().numpy()[0]
    except Exception as e:
        return None

# Build Knowledge Base
def build_knowledge_base(csv_path, img_dir, num_images=10, cache_path=None):
    """Build visual KB (same as before)"""

    if cache_path and os.path.exists(cache_path):
        print(f"Loading cached KB from {cache_path}...")
        with open(cache_path, 'rb') as f:
            kb = pickle.load(f)
        print(f"KB loaded: {len(kb)} species\n")
        return kb

    print(f"Building Knowledge Base...")
    df = pd.read_csv(csv_path)
    df['type'] = df['type'].replace('Inedible', 'Poisonous')

    kb = {}
    total_images = 0

    for idx, row in df.iterrows():
        species = row['species']
        label = row['type']
        species_dir = os.path.join(img_dir, species)

        if not os.path.exists(species_dir):
            continue

        image_files = [f for f in os.listdir(species_dir) if f.endswith('.png')]
        image_files = image_files[:num_images]

        features_list = []
        paths_list = []

        for img_file in image_files:
            img_path = os.path.join(species_dir, img_file)
            features = extract_image_features(img_path)

            if features is not None:
                features_list.append(features)
                paths_list.append(img_path)
                total_images += 1

        if features_list:
            kb[species] = {
                'label': label,
                'features': np.array(features_list),
                'paths': paths_list
            }

        if (idx + 1) % 20 == 0:
            print(f"   Processed {idx+1}/{len(df)}...")

    print(f"\nKB built: {len(kb)} species, {total_images} images")

    if cache_path:
        with open(cache_path, 'wb') as f:
            pickle.dump(kb, f)
        print(f"KB cached\n")

    return kb

# Build FAISS Index
def build_faiss_index(kb):
    """Build FAISS index"""
    print("Building FAISS index...")

    all_features = []
    metadata = []

    for species, data in kb.items():
        for img_idx, features in enumerate(data['features']):
            all_features.append(features)
            metadata.append((species, data['label'], img_idx))

    all_features = np.array(all_features).astype('float32')
    dimension = all_features.shape[1]
    index = faiss.IndexFlatIP(dimension)
    index.add(all_features)

    print(f"FAISS index built: {len(all_features)} vectors\n")
    return index, metadata

# RAG Retrieval
def retrieve_similar_examples(query_features, index, metadata, top_k=TOP_K):
    """Retrieve top-k similar (SAME AS STREAMLIT)"""
    query_features = query_features.reshape(1, -1).astype('float32')
    similarities, indices = index.search(query_features, top_k)

    results = []
    for sim, idx in zip(similarities[0], indices[0]):
        if idx >= len(metadata):
            continue
        species, label, img_idx = metadata[idx]
        results.append({
            'species': species,
            'label': label,
            'similarity': float(sim),
            'image_idx': img_idx
        })

    return results

# CLIP Prompt Prediction - SAME AS STREAMLIT
def predict_with_prompts(image_path):
    """Get CLIP prompt-based prediction"""
    try:
        image = Image.open(image_path).convert('RGB')
        all_prompts = EDIBLE_PROMPTS + POISONOUS_PROMPTS

        inputs = processor(
            text=all_prompts,
            images=image,
            return_tensors="pt",
            padding=True
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            probs = outputs.logits_per_image.softmax(dim=1).cpu().numpy()[0]

        edible_prob = probs[:len(EDIBLE_PROMPTS)].mean()
        poison_prob = probs[len(EDIBLE_PROMPTS):].mean()

        total = edible_prob + poison_prob
        edible_ratio = edible_prob / total if total > 0 else 0.5

        return edible_ratio
    except:
        return 0.5

# HYBRID PREDICTION - EXACT COPY FROM STREAMLIT
def predict_with_enhanced_rag(image_path, index, metadata):
    """
    Enhanced RAG prediction - EXACT SAME AS STREAMLIT
    """
    try:
        # Step 1: Extract features
        query_features = extract_image_features(image_path)
        if query_features is None:
            return "Unknown", 0.0, "Failed to extract features"

        # Step 2: Retrieve similar examples
        similar_examples = retrieve_similar_examples(query_features, index, metadata, top_k=TOP_K)

        if not similar_examples:
            return "Unknown", 0.0, "No KB data"

        # Step 3: Calculate RAG scores with distance weighting
        # CRITICAL: Use exponential decay (0.85^i) - SAME AS STREAMLIT
        label_scores = {'Edible': 0.0, 'Poisonous': 0.0}
        total_weight = 0.0

        for i, example in enumerate(similar_examples):
            # Distance-weighted scoring with exponential decay
            weight = example['similarity'] * (0.85 ** i)
            label_scores[example['label']] += weight
            total_weight += weight

        # Normalize RAG scores
        if total_weight > 0:
            rag_edible_score = label_scores['Edible'] / total_weight
        else:
            rag_edible_score = 0.5

        # Step 4: Get CLIP prompt score
        clip_edible_ratio = predict_with_prompts(image_path)

        # Step 5: Hybrid scoring (70% RAG + 30% CLIP)
        hybrid_edible_score = 0.7 * rag_edible_score + 0.3 * clip_edible_ratio

        # Step 6: Conservative safety logic - SAME AS STREAMLIT
        base_confidence = hybrid_edible_score

        if hybrid_edible_score >= SAFETY_THRESHOLD:
            classification = "Edible"
            confidence = hybrid_edible_score

            # Conservative check
            if base_confidence < MIN_CONFIDENCE:
                classification = "Poisonous"
                confidence = 1.0 - hybrid_edible_score
                reasoning = f"Low conf ({base_confidence:.1%}) → Conservative"
            else:
                reasoning = f"RAG: {rag_edible_score:.1%}, CLIP: {clip_edible_ratio:.1%}"
        else:
            classification = "Poisonous"
            confidence = 1.0 - hybrid_edible_score
            reasoning = f"Score: {hybrid_edible_score:.1%} < {SAFETY_THRESHOLD:.0%}"

        # Add similar species info
        top_species = [ex['species'] for ex in similar_examples[:3]]
        reasoning += f" | Similar: {', '.join(top_species[:2])}"

        return classification, confidence, reasoning

    except Exception as e:
        return "Unknown", 0.0, str(e)[:80]

# Evaluation
def run_evaluation(csv_path, img_dir, index, metadata, output_path, test_size=None):
    """Run evaluation with Hybrid RAG+CLIP"""

    print("EVALUATION - HYBRID RAG+CLIP (Matching Streamlit)")

    df = pd.read_csv(csv_path)
    df['type'] = df['type'].replace('Inedible', 'Poisonous')

    if test_size:
        df = df.head(test_size)
        print(f"\nTEST MODE: {test_size} samples\n")
    else:
        print(f"\nEvaluating {len(df)} samples\n")

    print(f"Settings:")
    print(f"  - TOP_K: {TOP_K}")
    print(f"  - SAFETY_THRESHOLD: {SAFETY_THRESHOLD}")
    print(f"  - Distance weighting: 0.85^i")
    print(f"  - Hybrid: 70% RAG + 30% CLIP\n")

    results = []
    start_time = time.time()

    for idx, row in df.iterrows():
        species = row['species']
        true_label = row['type']

        # Use first image
        img_path = os.path.join(img_dir, species, '1.png')

        if not os.path.exists(img_path):
            results.append([species, true_label, "Poisonous", 0.5, "Not found - safe default"])
            continue

        # Hybrid Prediction
        pred_label, confidence, reasoning = predict_with_enhanced_rag(img_path, index, metadata)

        results.append([species, true_label, pred_label, confidence, reasoning])

        # Progress
        correct = pred_label == true_label
        mark = "✅" if correct else "❌"

        if (idx + 1) % 20 == 0 or not correct:
            print(f"[{idx+1}/{len(df)}] {mark} {species}")
            print(f"  True: {true_label:<10} | Pred: {pred_label:<10} | Conf: {confidence:.1%}")

    elapsed = time.time() - start_time

    # Results DataFrame
    df_results = pd.DataFrame(
        results,
        columns=['species', 'true_label', 'predicted_label', 'confidence', 'reasoning']
    )
    df_results['correct'] = (df_results['true_label'] == df_results['predicted_label']).astype(int)
    df_valid = df_results[df_results['predicted_label'] != 'Unknown'].copy()

    # Print Results
    print("RESULTS")

    print(f"\nTime: {elapsed:.1f}s ({elapsed/60:.1f} min)")
    print(f"Speed: {len(df)/elapsed:.2f} img/s")

    if len(df_valid) > 0:
        acc = df_valid['correct'].mean()
        print(f"\nAccuracy: {acc:.2%} ({df_valid['correct'].sum()}/{len(df_valid)})")
        print(f"Avg Confidence: {df_valid['confidence'].mean():.1%}")

        # F1 Score
        f1 = f1_score(df_valid['true_label'], df_valid['predicted_label'], average='weighted')
        print(f"F1 Score (weighted): {f1:.4f}")

        # Confusion Matrix
        print("\nConfusion Matrix:")
        cm = confusion_matrix(df_valid['true_label'], df_valid['predicted_label'], labels=['Edible', 'Poisonous'])
        print(cm)
        print("\nLabels: ['Edible', 'Poisonous']")

        # Classification Report
        print("\nClassification Report:")
        print(classification_report(df_valid['true_label'], df_valid['predicted_label']))

        # Safety Analysis
        print("\nSAFETY ANALYSIS:")
        errors = df_valid[df_valid['correct'] == 0]

        false_edible = errors[(errors['predicted_label'] == 'Edible') & (errors['true_label'] == 'Poisonous')]
        false_poison = errors[(errors['predicted_label'] == 'Poisonous') & (errors['true_label'] == 'Edible')]

        print(f"  Total Errors: {len(errors)}")
        print(f"  FALSE EDIBLE (DANGEROUS): {len(false_edible)}")
        print(f"  FALSE POISONOUS (SAFE): {len(false_poison)}")

        if len(false_edible) > 0:
            print(f"\nMost dangerous errors:")
            for _, e in false_edible.sort_values('confidence', ascending=False).head(5).iterrows():
                print(f"     • {e['species']:<30} Conf: {e['confidence']:.1%}")

    # Save
    df_results.to_csv(output_path, index=False)
    print(f"\nResults saved: {output_path}")

    return df_results

# Main
if __name__ == "__main__":
    print("\nSTARTING HYBRID EVALUATION\n")

    try:
        # Step 1: Build KB
        print("STEP 1: BUILD KNOWLEDGE BASE")
        kb = build_knowledge_base(CSV_PATH, IMG_DIR, NUM_KB_IMAGES, KB_CACHE_PATH)

        # Step 2: Build FAISS Index
        print("STEP 2: BUILD FAISS INDEX")
        index, metadata = build_faiss_index(kb)

        # Step 3: Run Evaluation
        print("STEP 3: RUN EVALUATION\n")

        TEST_SIZE = None  # None = all

        results_df = run_evaluation(
            CSV_PATH, IMG_DIR, index, metadata, OUTPUT_PATH, test_size=TEST_SIZE
        )

        print("EVALUATION COMPLETE!")

        print(f"\n This evaluation matches Streamlit production code:")
        print(f"   • Same TOP_K: {TOP_K}")
        print(f"   • Same distance weighting: 0.85^i")
        print(f"   • Same hybrid: 70% RAG + 30% CLIP")
        print(f"   • Same safety threshold: {SAFETY_THRESHOLD}")
        print(f"   • Same conservative logic")

    except Exception as e:
        print(f"\nERROR: {e}")
        import traceback
        traceback.print_exc()

Device: cpu

Loading openai/clip-vit-large-patch14...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Model loaded


STARTING HYBRID EVALUATION

STEP 1: BUILD KNOWLEDGE BASE
Loading cached KB from /content/drive/MyDrive/mushroom_knowledge_base.pkl...
KB loaded: 215 species

STEP 2: BUILD FAISS INDEX
Building FAISS index...
FAISS index built: 2137 vectors

STEP 3: RUN EVALUATION

EVALUATION - HYBRID RAG+CLIP (Matching Streamlit)

Evaluating 215 samples

Settings:
  - TOP_K: 3
  - SAFETY_THRESHOLD: 0.55
  - Distance weighting: 0.85^i
  - Hybrid: 70% RAG + 30% CLIP

[5/215] ❌ aniseed_funnel
  True: Edible     | Pred: Poisonous  | Conf: 66.7%
[14/215] ❌ bitter_bolete
  True: Poisonous  | Pred: Edible     | Conf: 94.8%
[20/215] ✅ blue_roundhead
  True: Poisonous  | Pred: Poisonous  | Conf: 55.0%
[21/215] ❌ blushing_bracket
  True: Poisonous  | Pred: Edible     | Conf: 70.5%
[24/215] ❌ bovine_bolete
  True: Edible     | Pred: Poisonous  | Conf: 48.9%
[39/215] ❌ common_bonnet
  True: Edible     | Pred: Poisonous  | Conf: 47.5%
[40/215] ✅ common_inkcap
  True: Poisonous  | Pred: Poisonous  | C